# Inspect DeepPhys Models & Pre-Trained Checkpoints by Different Authors

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
import sys
sys.path.append('../')
from torch.utils.tensorboard import SummaryWriter

## Implementation by terbed/Deep-rPPG

In [7]:
from models.DeepPhys import DeepPhysConfig, DeepPhys

img_h, img_w = 36, 36
model_args = dict(
    img_h=img_h,
    img_w=img_w,
    out_dim=1,
    bias=True,
    dropout=0.50,
    nir_imgs_mean=0.0,
    nir_imgs_std=1.0,
    rppg_labels_diff_std=6.969092845916748
)

model_config = DeepPhysConfig(**model_args)
model = DeepPhys(model_config)

batch_size = 1
summary(model, input_size=[(batch_size, 2, 1, img_h, img_w), (batch_size, 2, 1)], device='cpu',
        col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds", "trainable"), verbose=2,
        depth=4, row_settings=("depth", "var_names"));

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds                 Trainable
DeepPhys (DeepPhys)                      [1, 2, 1, 36, 36]         [1, 1]                    --                        --                        --                        True
├─Conv2d (a_conv1): 1-1                  [1, 1, 36, 36]            [1, 32, 36, 36]           320                       [3, 3]                    414,720                   True
│    └─weight                                                                                ├─288                     [1, 32, 3, 3]
│    └─bias                                                                                  └─32                      [32]
├─BatchNorm2d (a_bn1): 1-2               [1, 32, 36, 36]           [1, 32, 36, 36]           64                        --                        64                        True
│    └─weight                     

In [4]:
writer = SummaryWriter()
nir_imgs = torch.randn(batch_size, 2, 1, img_h, img_w)
rppg_labels = torch.randn(batch_size, 2, 1)
writer.add_graph(model, [nir_imgs, rppg_labels])
writer.close()